# Crea un agente RAG con LangChain.

Estudiante: Camilo Andrés Quintero Rodríguez

## 1. Instalación y configuración del entorno

Se instalan las dependencias necesarias para trabajar con **LangChain**, los modelos de lenguaje y el procesamiento de texto:

```bash
pip install langchain langchain-text-splitters langchain-community bs4


## 2. Configuración de variables de entorno para LangSmith

En este paso se configuran las **variables de entorno** necesarias para habilitar el seguimiento y monitoreo de las ejecuciones del agente dentro de **LangSmith**, una herramienta de observabilidad para LangChain.  

El código usa el módulo `os` para establecer valores que el sistema puede leer durante la ejecución del programa.

```bash
pip install langchain langsmith
pip install -U langchain langsmith

In [ ]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = ""

## 3. Inicialización del modelo de lenguaje (LLM)

En este paso se configura el **modelo de lenguaje** que el agente RAG utilizará para generar y razonar sobre las respuestas.  
Se usa el modelo **Gemini 2.5 Flash Lite** de Google, accedido a través de la integración de **LangChain**.

```bash
pip install langchain-google-genai

In [ ]:
import os
from langchain.chat_models import init_chat_model

os.environ["GOOGLE_API_KEY"] = ""

model = init_chat_model("google_genai:gemini-2.5-flash-lite")

## 4. Creación del almacén vectorial y configuración de embeddings

En este paso se prepara el sistema para **almacenar y recuperar información semántica**.  
Esto es fundamental en un agente RAG (Retrieval-Augmented Generation), ya que permite **buscar fragmentos relevantes de texto** antes de generar una respuesta.

```bash
pip install langchain langchain-community langchain-text-splitters langchain-google-genai bs4

In [3]:
import os
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector_store = InMemoryVectorStore(embeddings)

USER_AGENT environment variable not set, consider setting it to identify your requests.


## 5. Carga y limpieza del documento fuente

En este paso se **extrae contenido de una página web** para usarlo como base de conocimiento del agente RAG.  
Se utiliza el cargador `WebBaseLoader` de **LangChain Community**, junto con **BeautifulSoup (bs4)** para limpiar el HTML y conservar solo la información relevante del artículo.

```bash
pip install langchain-community bs4

In [4]:
# step1_load_data.py
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Solo extrae el contenido relevante de la página
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))

# Cargar el artículo del blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Descargar y limpiar el contenido
docs = loader.load()

# Validar que se haya cargado correctamente
assert len(docs) == 1, "No se pudo cargar el documento desde la URL."

# Mostrar información básica
print(f"✅ Documento cargado correctamente.")
print(f"📄 Total de caracteres extraídos: {len(docs[0].page_content)}")

# (opcional) Muestra los primeros 500 caracteres
print("\n🔹 Fragmento del texto extraído:")
print(docs[0].page_content[:500])


✅ Documento cargado correctamente.
📄 Total de caracteres extraídos: 43047

🔹 Fragmento del texto extraído:


      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


## 6. División del texto en fragmentos (chunks)

En este paso se divide el texto cargado en **fragmentos más pequeños**, lo cual es esencial para el funcionamiento del modelo RAG.  
Los modelos de lenguaje y los sistemas de búsqueda vectorial trabajan mejor cuando el texto está segmentado en partes manejables.

```bash
pip install langchain-text-splitters

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


## 7. Creación de embeddings y almacenamiento vectorial

En este paso se **convierte cada fragmento de texto en un vector numérico** (embedding) y se almacena en una base de datos vectorial en memoria.  
Esto permite que el agente RAG busque información relevante de forma eficiente mediante **búsqueda semántica**.

```bash
pip install sentence-transformers langchain-community

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = InMemoryVectorStore(embeddings) 
document_ids = vector_store.add_documents(documents=all_splits)


C:\Users\Milo\AppData\Local\Temp\ipykernel_68\105888337.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Milo\Documents\RAG-LangChain\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Milo\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 8. Verificación de los documentos almacenados

Para confirmar que los fragmentos de texto fueron correctamente convertidos en vectores y guardados en la base de datos, se imprime una muestra de los **IDs** generados:


In [10]:
print(document_ids[:3])

['bd2df2a3-d978-4aad-b579-8d4239761924', 'e52acb44-c86d-47d5-9789-bcbe456912cd', '553e5c8b-267e-4726-a638-b4533722f02c']


## 9. Creación de una herramienta para recuperar contexto

En este paso se define una función que permitirá buscar información relevante dentro de los documentos vectorizados, con el objetivo de **proveer contexto** a las consultas que se realicen al modelo.


In [11]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

## 10. Creación del agente RAG

En este paso se crea el **agente inteligente (RAG)** que usará el modelo de lenguaje junto con la herramienta de búsqueda para responder preguntas basadas en el contenido del documento cargado.


In [12]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

## 11. Consulta al agente

En este paso se realiza una **pregunta al agente** para verificar su funcionamiento.  
El agente debe usar la herramienta de recuperación de contexto para buscar información relevante en el texto previamente cargado y responder con base en ella.


In [13]:
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (5412a908-164c-453e-9894-df915905533f)
 Call ID: 5412a908-164c-453e-9894-df915905533f
  Args:
    query: standard method for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on a

## 12. Incorporar contexto dinámico al agente

En este paso se implementa una **middleware** que añade contexto de manera automática a las consultas del usuario.  
Esto mejora la precisión del modelo, ya que cada vez que se formula una pregunta, el agente busca información relevante en la base de documentos y la incorpora al *prompt* antes de generar la respuesta.


In [14]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(model, tools=[], middleware=[prompt_with_context])

## 13. Prueba del agente con contexto dinámico

En este paso se pone a prueba el nuevo agente, que ahora incorpora automáticamente contexto relevante de la base de conocimiento cada vez que recibe una pregunta.


In [15]:
query = "What is task decomposition?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is task decomposition?
================================== Ai Message ==================================

Task decomposition is the process of breaking down a complex task into smaller, more manageable sub-tasks. This is a crucial step in planning and executing complicated actions, as it makes the overall objective easier to understand and achieve.

There are several ways task decomposition can be performed:

*   **By LLM with simple prompting:** This involves using prompts like "Steps for XYZ" or "What are the subgoals for achieving XYZ?" to guide a Large Language Model (LLM) to break down a task.
*   **By using task-specific instructions:** For certain types of tasks, specific instructions can be given. For example, for writing a novel, an instruction like "Write a story outline" would initiate task decomposition.
*   **With human inputs:** A human can directly provide the decomposed steps for a tas